# Genetic Programming for Feature Engineering
- <a href=#tree>Function Tree Class and Function Definitions</a>
- <a href=#GP>Genetic Programming Functions Definitions</a>
- <a href=#sample>Apply some Sample Trees to a Dataframe</a>
- <a href=#test>Apply the GP to a Test dataset</a>

<a id=top></a>
- TODO: can I fix the power() function back to properly handling negative exponents (for a^b, a should be float)?
- TODO: can I fix tree printing?


In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import ipdb
from collections import OrderedDict
import time
from itertools import chain
import copy
import re
import warnings

import chart_studio.plotly as ply
import chart_studio.tools as plytool
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as plyoff
import plotly.subplots as plysub

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)

## Generically-useful Functions

In [ ]:
''' generic function to perform weighted random selection '''
def RandomWeightedSelect(keys, wats, randSeed=None):
    '''
    Randomly select an item from a list, according to a set of
    specified weights.
    :param keys: array-like of items from which to select
    :param wats: array-like of weights associated with the input
        keys; must be sorted in descending weight
    :param randSeed: optional random seed for np.random; if no
        randomization is desired, pass 0
    :return selection: selected item
    :return randSeed: random seed used
    '''
    
    # ranodmize, perhaps
    if randSeed != 0:
        if randSeed is None:
            randSeed = int(str(time.time()).split('.')[1])
        np.random.seed(randSeed)
    
    # ensure weights sum to 1
    totWats = sum(wats)
    if totWats != 1:
        wats = [v/totWats for v in wats]
    
    # get the cumulative weights
    cumWats = np.cumsum(wats)
    # get the indices of where the random [0,1] is < the cum weight
    rnd = np.random.rand()
    seld = rnd < cumWats
    
    return [k for (k,s) in zip(keys, seld) if s][0], randSeed

In [ ]:
''' binary arithmetic operations that can be called as functions '''
# addition
def ad(a, b):
    return a+b

# subtraction
def sb(a, b):
    return a-b

# multiplication
def ml(a, b):
    return np.nan_to_num(a * b, posinf=np.nan)

# division
def dv(a, b):
    # check for longest dimensional match
    try:
        lna = len(a)
    except TypeError:
        lna = 1
    try:
        lnb = len(b)
    except TypeError:
        lnb = 1
    # compute
    if (lna == lnb) & (lna == 1):
        # both scalars
        if b != 0:
            res = a/b
        else:
            res = np.nan
    else:
        # at least 1 iterable
        if lna < lnb:
            # a is scalar, b is not
            a = [a]*lnb
        elif lnb < lna:
            # b is scalar, a is not
            b = [b]*lna
        res = np.nan_to_num(a / b, posinf=np.nan)
    return res

# power
def pw(a, b):
    return np.nan_to_num(a ** np.abs(b), posinf=np.nan)

# minimum
def mn(a, b):
    # check for longest dimensional match
    try:
        lna = len(a)
    except TypeError:
        lna = 1
    try:
        lnb = len(b)
    except TypeError:
        lnb = 1
    # compute
    if (lna == lnb) & (lna == 1):
        # both scalars
        res = min(a, b)
    elif lna == lnb:
        # both iterables
        res = np.where(a < b, a, b)
    elif lna < lnb:
        # a is scalar, b is not
        tmp = [a]*lnb
        res = np.where(tmp < b, tmp, b)
    elif lnb < lna:
        # b is scalar, a is not
        tmp = [b]*lna
        res = np.where(a < tmp, a, tmp)
    return res

# maximum
def mx(a, b):
    # check for longest dimensional match
    try:
        lna = len(a)
    except TypeError:
        lna = 1
    try:
        lnb = len(b)
    except TypeError:
        lnb = 1
    # compute
    if (lna == lnb) & (lna == 1):
        # both scalars
        res = max(a, b)
    elif lna == lnb:
        # both iterables
        res = np.where(a > b, a, b)
    elif lna < lnb:
        # a is scalar, b is not
        tmp = [a]*lnb
        res = np.where(tmp > b, tmp, b)
    elif lnb < lna:
        # b is scalar, a is not
        tmp = [b]*lna
        res = np.where(a > tmp, a, tmp)
    return res

In [ ]:
def ResultsPlots(data, sequenceCol, responseCol, predCol, resdCol, colorCol, overall_title, plot_colors=('red',)*4):
    '''
    This creates and returns a quad-plot of results from a model. The four plots are: a) prediction vs response,
    b) histogram of residuals, c) residuals by sequence, d) residuals by response. They are arranged as
    [[a,b],[c,d]].
    :param data: dataframe holding the sequence, response, prediction, and residual columns
    :param sequenceCol: column in the dataframe holding the time or sequence counter; if None, a counter will be used
    :param responseCol: column in the dataframe holding the response variable
    :param predCol: column in the dataframe holding the model predictions
    :param resdCol: column in the dataframe holding the model residuals
    :param colorCol: optional column in the dataframe holding the color for each observation for the plots
    :param overall_title: title to go on top of the quad plot
    :param plot_colors (default=(red, red, red, red)): optional tuple of colors for each plot; only used if colorCol
        not passed
    :return fig: plotly plot figure
    '''
    
    # copy the input dataframe
    data = data.copy(deep=True)
    
    if colorCol is None:
        colorCol = 'NOCOLOR'
    
    # setup the subplot
    figRes = plysub.make_subplots(rows=2, cols=2, subplot_titles=['Predictions vs Responses', 'Residuals Distribution','Residuals by Sequence', 'Residuals vs Responses'])
    
    # for the actual vs preds plot, build the fit line
    actual = data[responseCol].values.reshape(-1,1)
    lindat = np.linspace(actual.min(), actual.max(), 10).reshape(-1, 1)
    fitlin = LinearRegression(n_jobs=-1)
    fitlin.fit(X=actual, y=data[predCol])
    actpred = fitlin.predict(X=lindat)
    r2 = fitlin.score(X=actual, y=data[predCol])
    # create the trace and annotation
    r2trc = go.Scatter(x=lindat.squeeze(), y=actpred, mode='lines', name='fit', line={'color':'black','width':1}, showlegend=False)
    r2ann = dict(x=lindat[5][0], y=actpred[5], xref='x1', yref='y1', text='$\\rho=%0.3f$'%(r2), showarrow=False, bgcolor='#ffffff')
    
    # actuals vs resids plot
    if colorCol == 'NOCOLOR':
        data[colorCol] = plot_colors[0]
    figRes.add_trace(go.Scatter(x=data[responseCol], y=data[predCol], mode='markers', marker={'color':data[colorCol]}, showlegend=False), 1,1)
    figRes.add_trace(r2trc, 1, 1)
    figRes['layout']['xaxis1'].update(title=responseCol)
    figRes['layout']['yaxis1'].update(title=predCol)
    
    # residuals histogram
    if colorCol == 'NOCOLOR':
        data[colorCol] = plot_colors[1]
    figRes.add_trace(go.Histogram(x=data[resdCol], histnorm='', marker={'color':data[colorCol]}, showlegend=False), 1,2)
    figRes['layout']['xaxis2'].update(title=resdCol)
    figRes['layout']['yaxis2'].update(title='count')
    
    # get the time variable
    if sequenceCol is None:
        seq = list(range(len(data)))
        seqNam = 'sequence'
    else:
        seq = data[sequenceCol]
        seqNam = sequenceCol
    
    # residuals by time plot
    if colorCol == 'NOCOLOR':
        data[colorCol] = plot_colors[2]
    figRes.add_trace(go.Scatter(x=seq, y=data[resdCol], mode='markers', marker={'color':data[colorCol]}, showlegend=False), 2, 1)
    figRes['layout']['xaxis3'].update(title=seqNam)
    figRes['layout']['yaxis3'].update(title=resdCol)
    
    # residuals by response plot
    if colorCol == 'NOCOLOR':
        data[colorCol] = plot_colors[3]
    figRes.add_trace(go.Scatter(x=data[responseCol], y=data[resdCol], mode='markers', marker={'color':data[colorCol]}, showlegend=False), 2,2)
    figRes['layout']['xaxis4'].update(title=responseCol)
    figRes['layout']['yaxis4'].update(title=resdCol)
    
    # update layout
    figRes['layout'].update(title=overall_title, height=1000)
    anns = list(figRes['layout']['annotations'])
    anns.append(r2ann)
    figRes['layout']['annotations'] = anns
    
    return figRes

## Function Tree Class and Function Definitions
<a id=tree></a>
<a href=#top>Go to Top</a>

In [ ]:
''' a node encapsulates an operation / fatures / constant; it remembers it's parent, and knows it's children '''
class Node(object):
    def __init__(self, typ, val, par):
        self.type = typ
        self.value = val
        self.parent = par
        # init the string and left & right to nothing
        self.Str = '%s(%s)'%(self.type, self.value)
        self.left = None
        self.right = None
        self.setStr('LR')
        
    def __str__(self):
        return '(%s) -> [%s, %s]'%(self.Str, self.leftStr, self.rightStr)
    
    def setLeft(self, L):
        self.left = L
        self.setStr('L')
        
    def setRight(self, R):
        self.right = R
        self.setStr('R')
    
    def setStr(self, LR):
        if 'L' in LR:
            if self.left == None:
                self.leftStr = '_'
            else:
                self.leftStr = '%s(%s)'%(self.left.type, self.left.value)
        if 'R' in LR:
            if self.right == None:
                self.rightStr = '_'
            else:
                self.rightStr = '%s(%s)'%(self.right.type, self.right.value)

In [ ]:
''' a tree represents the entire function; it knows the root, level-ordered structure, location of leaves, and depth '''
class Tree(object):
    def __init__(self, root, maxDepth):
        self.root = root
        self.depth = maxDepth # init actual depth with just the max depth allowed for now
        self.leaves = None
        self.struct = None
        self.function = None
        # build the structure dict & function string
        self.GenStruct()
        
    def __str__(self):
        # first build the struct dict, if necessary
        if self.struct is None:
            self.GenStruct()
        # now print
        return '\n'.join(['%d: %s'%(key, '|'.join([str(node.value) for node in val])) for (key, val) in self.struct.items()])
    
    def __repr__(self):
        return self.__str__()

    @staticmethod
    def __RecTreeStruct(currNode, tree, leaves, currKey):
        '''
        Recursive tree structuring function; only to be called by TreeStruct
        '''
        # save the node
        this = tree[currKey].copy()
        this.append(currNode)
        tree[currKey] = this
        
        # save the node if it's a leaf
        if currNode.type in ['const', 'feat']:
            this = leaves[currKey].copy()
            this.append(currNode)
            leaves[currKey] = this
            
        if (currNode.left is None) & (currNode.right is None):
            return tree, leaves

        if currNode.left is not None:
            tree, leaves = Tree.__RecTreeStruct(currNode.left, tree, leaves, currKey+1)
        if currNode.right is not None:
            tree, leaves = Tree.__RecTreeStruct(currNode.right, tree, leaves, currKey+1)
        return tree, leaves

    def GenStruct(self):
        '''
        Return the function tree structure as a dictionary.
        :return tree: level number-keyed ordered dict of the tree
        '''
        # populate the tree view dict; have to init the dict to a large number, because
        # with crossovers, trees could get very large
        self.struct = dict.fromkeys(range(1000), [])
        self.leaves = dict.fromkeys(range(1000), [])
        self.struct, self.leaves = Tree.__RecTreeStruct(self.root, self.struct, self.leaves, 0)
        # prune now (remove unused rows)
        for key in list(self.struct.keys()):
            if self.struct[key] == []:
                self.struct.pop(key)
        for key in list(self.leaves.keys()):
            if self.leaves[key] == []:
                self.leaves.pop(key)
        
        # set the depth
        self.depth = max(self.struct.keys())+1
        
        # now generate the function
        self.function = self.GenFunction()
        
        return None

    def GenFunction(self):
        '''
        Returns a string representation of the function tree as
        a function. This can also act as the hash of a tree.
        :return function: the string of the function
        '''
        
        funcStrings = {}

        # special handling of const or feat root nodes
        if self.root.type != 'op':
            funcStrings[self.root] = str(self.root.value)
        else:
            # start at the top & climb down the tree
            for currLev in range(self.depth-1, 0, -1):
                nodes = self.struct[currLev]
                # parse the nodes at this level and iterate in pairs
                for indx in range(0, len(nodes), 2):
                    # if there's a func string already defined, use them
                    lVal = funcStrings.get(nodes[indx], str(nodes[indx].value))
                    rVal = funcStrings.get(nodes[indx+1], str(nodes[indx+1].value))
                    # build and store the function string
                    funcStrings[nodes[indx].parent] = nodes[indx].parent.value + '(' + lVal + ',' + rVal + ')'
                    
        return funcStrings[self.root]

    def __Simp(self):
        '''
        Simplify trees by collapsing nodes that have: both children are numbers,
        multiplication / subtraction / addition / power with 0, 1 to a power,
        multiplication by 1, power / division by 1, min / max / subtraction
        with identical children. Must be called, ideally, from Simplify().
        '''
        
        # prepare the iteraion
        levs = list(self.leaves.keys())
        levs.sort
        levs = levs[::-1]
        try:
            levs.remove(0)
        except ValueError:
            # no 0, so just ignore
            pass
        # iterate over each non-root level, and iterate over each leaf to check
        # if it's parent can be simplified; this moves some leaves, and I'm unsure
        # yet of the consequences
        delta = False
        for lev in levs:
            for leaf in self.leaves[lev]:
                # both siblings just numbers - replace the parent, but not if dividing by 0
                if (leaf.parent.left.type == 'const') & (leaf.parent.right.type == 'const') & ~((leaf.parent.value == 'dv') & (leaf.parent.right.value == 0)):
                    # reduce the parent to a new const node
                    newConst = eval(Tree(leaf.parent, 2).GenFunction())
                    leaf.parent.type = 'const'
                    leaf.parent.value = newConst
                    # clear children
                    leaf.parent.setLeft(None)
                    leaf.parent.setRight(None)
                    # remember that we made a change
                    delta = True
                    break
                # mult by 1
                elif (leaf.value == 1) & (leaf.parent.value == 'ml'):
                    # reduce the parent to whichever sibling is not 1, but first check if the parent is a root node:
                    if leaf.parent == self.root:
                        # parent is root, so need to edit the root
                        if leaf.parent.left.value != 1:
                            # point to the left sibling
                            self.root = leaf.parent.left
                            leaf.parent.left.parent = None
                        elif leaf.parent.right.value != 1:
                            # point to the right sibling
                            self.root = leaf.parent.right
                            leaf.parent.right.parent = None
                    else:
                        if leaf.parent.left.value != 1:
                            # figure out if the parent is a left or right
                            if leaf.parent == leaf.parent.parent.left:
                                # point to the left sibling
                                leaf.parent.parent.setLeft(leaf.parent.left)
                            else:
                                # point to the left sibling
                                leaf.parent.parent.setRight(leaf.parent.left)
                            leaf.parent.left.parent = leaf.parent.parent
                        elif leaf.parent.right.value != 1:
                            # figure out of the parent is a left or right
                            if leaf.parent == leaf.parent.parent.left:
                                # point to the right sibling
                                leaf.parent.parent.setLeft(leaf.parent.right)
                            else:
                                # point to the right sibling
                                leaf.parent.parent.setRight(leaf.parent.right)
                            leaf.parent.right.parent = leaf.parent.parent
                    # remember that we made a change
                    delta = True
                    break
                # div / pow by 1 (1 on the right)
                elif (leaf.value == 1) & (leaf.parent.value in ['pw', 'dv'])  & (leaf.parent.right == leaf):
                    # reduce the parent to the left sibling, but first check if the parent is a root node:
                    if leaf.parent == self.root:
                        # parent is root, so need to edit the root
                        self.root = leaf.parent.left
                        leaf.parent.left.parent = None
                    else:
                        # figure out if the parent is a left or right
                        if leaf.parent == leaf.parent.parent.left:
                            # point to the left sibling
                            leaf.parent.parent.setLeft(leaf.parent.left)
                        else:
                            # point to the left sibling
                            leaf.parent.parent.setRight(leaf.parent.left)
                        # set the leaf's parent
                        leaf.parent.left.parent = leaf.parent.parent
                    # remember that we made a change
                    delta = True
                    break
                # pow by 1 (1 on the left)
                elif (leaf.value == 1) & (leaf.parent.value == 'pw')  & (leaf.parent.left == leaf):
                    # just replace with constant 1
                    leaf.parent.type = 'const'
                    leaf.parent.value = 1
                    # clear children
                    leaf.parent.setLeft(None)
                    leaf.parent.setRight(None)
                    # remember that we made a change
                    delta = True
                    break
                # mult by 0
                elif (leaf.value == 0) & (leaf.parent.value == 'ml'):
                    # just replace with constant 0
                    leaf.parent.type = 'const'
                    leaf.parent.value = 0
                    # remove leaves
                    try:
                        self.leaves[lev].remove(leaf.parent.left)
                        self.leaves[lev].remove(leaf.parent.right)
                    except ValueError:
                        pass
                    # clear children
                    leaf.parent.setLeft(None)
                    leaf.parent.setRight(None)
                    # remember that we made a change
                    delta = True
                    break
                # pow by 0
                elif (leaf.value == 0) & (leaf.parent.value == 'pw'):
                    # just replace with constant 0 or 1 depending on the side
                    if leaf.parent.left == leaf:
                        leaf.parent.value = 0
                    elif leaf.parent.right == leaf:
                        leaf.parent.value = 1
                    leaf.parent.type = 'const'
                    # clear children
                    leaf.parent.setLeft(None)
                    leaf.parent.setRight(None)
                    # remember that we made a change
                    delta = True
                    break
                #  add 0
                elif (leaf.value == 0) & (leaf.parent.value == 'ad'):
                    # reduce the parent to whichever sibling is not 0, but first check if the parent is a root node:
                    if leaf.parent == self.root:
                        # parent is root, so need to edit the root
                        if leaf.parent.left.value != 0:
                            # point to the left sibling
                            self.root = leaf.parent.left
                            leaf.parent.left.parent = None
                        elif leaf.parent.right.value != 0:
                            # point to the right sibling
                            self.root = leaf.parent.right
                            leaf.parent.right.parent = None
                    else:
                        if leaf.parent.left.value != 0:
                            # figure out of the parent is a left or right
                            if leaf.parent == leaf.parent.parent.left:
                                # point to the left sibling
                                leaf.parent.parent.setLeft(leaf.parent.left)
                            else:
                                # point to the left sibling
                                leaf.parent.parent.setRight(leaf.parent.left)
                            leaf.parent.left.parent = leaf.parent.parent
                        elif leaf.parent.right.value != 0:
                            # figure out of the parent is a left or right
                            if leaf.parent == leaf.parent.parent.left:
                                # point to the left sibling
                                leaf.parent.parent.setLeft(leaf.parent.right)
                            else:
                                # point to the left sibling
                                leaf.parent.parent.setRight(leaf.parent.right)
                            leaf.parent.right.parent = leaf.parent.parent
                    # remember that we made a change
                    delta = True
                    break
                #  subtract 0 (0 on right)
                elif (leaf.value == 0) & (leaf.parent.value == 'sb') & (leaf.parent.right == leaf):
                    # reduce the parent to the left sibling, but first check if the parent is a root node:
                    if leaf.parent == self.root:
                        # parent is root, so need to edit the root
                        self.root = leaf.parent.left
                        leaf.parent.left.paretn = None
                    else:
                        # figure out of the parent is a left or right
                        if leaf.parent == leaf.parent.parent.left:
                            # point to the left sibling
                            leaf.parent.parent.setLeft(leaf.parent.left)
                        else:
                            # point to the left sibling
                            leaf.parent.parent.setRight(leaf.parent.left)
                        # set the leaf's parent
                        leaf.parent.left.parent = leaf.parent.parent
                    # remember that we made a change
                    delta = True
                # min, max, or sub the same inputs
                elif (leaf.parent.left.value == leaf.parent.right.value) & (leaf.type != 'op') & (leaf.parent.value in ['mn', 'mx', 'sb']):
                    if leaf.parent.value == 'sb':
                        # just replace with constant 0
                        leaf.parent.type = 'const'
                        leaf.parent.value = 0
                        # remove leaves
                        try:
                            self.leaves[lev].remove(leaf.parent.left)
                            self.leaves[lev].remove(leaf.parent.right)
                        except ValueError:
                            pass
                        # clear children
                        leaf.parent.setLeft(None)
                        leaf.parent.setRight(None)
                    else:
                        # replace parent with whatever the siblings are, but first check if the parent is a root node:
                        if leaf.parent == self.root:
                            self.root = leaf
                            leaf.parent = None
                        else:
                            if leaf.parent == leaf.parent.parent.left:
                                # point to the left sibling
                                leaf.parent.parent.setLeft(leaf)
                            else:
                                # point to the left sibling
                                leaf.parent.parent.setRight(leaf)
                            # set the leaf's parent
                            leaf.parent = leaf.parent.parent
                    # remember that we made a change
                    delta = True
                    break
            
            # see if we made any changes
            if delta:
                break
                                
        return delta
    
    def Simplify(self, maxSimp=10):
        '''
        Simplify trees by collapsing nodes that have: both children are numbers,
        multiplication / subtraction / addition / power with 0, 1 to a power,
        multiplication by 1, power / division by 1, min / max / subtraction
        with identical children. This will make several passes of the tree, with
        each pass resulting in at most 1 simplifying change, followed by a call to
        GenStruct().
        :param maxSimp: optional (default=10) maximum number of simplification attempts
        :return sCount: number of simplification attempts made
        '''
        
        # iterate over attemps allowed
        for sCount in range(maxSimp):
            # simplify
            delta = self.__Simp()
            if delta:
                # rebuild
                self.GenStruct()
            else:
                break
                
        return sCount+1

In [ ]:
''' functions for building a new tree '''
def BuildTreeRec(currNode, currDepth, maxDepth, nodeMeta):
    '''
    Recursive tree building function; only to be called by BuildTree
    '''

    # exit if too deep or at a leaf
    if (currDepth == maxDepth) or (currNode.type != 'op'):
        return currNode
    
    # hit one short of max depth, so ensure only consts or feats selected
    if currDepth == (maxDepth-1):
        noOpsK = [k for k in nodeMeta.keys() if k != 'op']
        noOpsW = [nodeMeta[t][2] for t in noOpsK]
        nodeTypeL, _ = RandomWeightedSelect(noOpsK, noOpsW, 0)
        nodeTypeR, _ = RandomWeightedSelect(noOpsK, noOpsW, 0)
    else:
        nodeTypeL, _ = RandomWeightedSelect(nodeMeta.keys(), [v[2] for v in nodeMeta.values()], 0)
        nodeTypeR, _ = RandomWeightedSelect(nodeMeta.keys(), [v[2] for v in nodeMeta.values()], 0)
        
    # randomly generate the left node
    nodeValuL = nodeMeta[nodeTypeL][0][np.random.randint(nodeMeta[nodeTypeL][1])]
    nodeL = BuildTreeRec(Node(nodeTypeL, nodeValuL, currNode), currDepth+1, maxDepth, nodeMeta)
    currNode.setLeft(nodeL)

    # randomly generate the right node
    nodeValuR = nodeMeta[nodeTypeR][0][np.random.randint(nodeMeta[nodeTypeR][1])]
    nodeR = BuildTreeRec(Node(nodeTypeR, nodeValuR, currNode), currDepth+1, maxDepth, nodeMeta)
    currNode.setRight(nodeR)
    
    return currNode

def BuildTree(maxDepth, nodeMeta, verbose=False):
    '''
    Using a set of types of nodes, build a functional tree.
    :param maxDepth: integer maximum depth allowed for the tree (including the root)
    :param nodeMeta: dictionary holding the a tuple of a list of the node values
        allowed, the number of node values allowed, and node weight for random
        selection; keys are node types of 'ops, 'feat', and 'const'
    :param verbose: optional (default = false) flag to print some info
    :return tree: the complete functional tree
    '''
    
    # if max depth is 1, can't have ops nodes
    if maxDepth == 1:
        # randomly generate the root node type
        noOpsK = [k for k in nodeMeta.keys() if k != 'op']
        noOpsW = [nodeMeta[t][2] for t in noOpsK]
        nodeType, _ = RandomWeightedSelect(noOpsK, noOpsW, 0)
    else:
        # randomly generate the root node type
        nodeType, _ = RandomWeightedSelect(nodeMeta.keys(), [v[2] for v in nodeMeta.values()], 0)
    
    # randomly generate the root node value
    nodeValu = nodeMeta[nodeType][0][np.random.randint(nodeMeta[nodeType][1])]    
    # build the tree
    rootNode = BuildTreeRec(Node(nodeType, nodeValu, None), 0, maxDepth-1, nodeMeta)
    
    return Tree(rootNode, maxDepth)

## Genetic Programming Function Definitions
<a id=GP></a>
<a href=#top>Go to Top</a>

In [ ]:
def TreesCrossover(this, that, verbose=False):
    '''
    Cross two trees at a node selected at random.
    :param this: first tree to cross
    :param that: second tree to cross
    :param verbose: optional (default=False) flag to print the crossover node
    :return this_new: new crossed-over tree
    :return that_new: new crossed-over tree
    '''
    
    # first create copies of the input trees
    thisC = copy.deepcopy(this)
    thatC = copy.deepcopy(that)
    
    # get the random crossover points
    thisXoverNode = np.random.permutation(list(chain.from_iterable(thisC.struct.values())))[0]
    if verbose:
        print('This crossover point: %s '%thisXoverNode)
    thatXoverNode = np.random.permutation(list(chain.from_iterable(thatC.struct.values())))[0]
    if verbose:
        print('That crossover point: %s '%thatXoverNode)
    
    # reassign the children
    try:
        if thisXoverNode.parent.right == thisXoverNode:
            thisXoverNode.parent.setRight(thatXoverNode)
        elif thisXoverNode.parent.left == thisXoverNode:
            thisXoverNode.parent.setLeft(thatXoverNode)
    except AttributeError:
        # this is a root node, so there is no parent, so no left or right
        pass
    try:    
        if thatXoverNode.parent.right == thatXoverNode:
            thatXoverNode.parent.setRight(thisXoverNode)
        elif thatXoverNode.parent.left == thatXoverNode:
            thatXoverNode.parent.setLeft(thisXoverNode)
    except AttributeError:
        # that is a root node, so there is no parent, so no left or right
        pass
    
    # reassign the parents
    thisXoverNode.parent, thatXoverNode.parent = thatXoverNode.parent, thisXoverNode.parent
    
    # if either is an orphan from having been crossed with a root, make it a new tree;
    # otherwise, just rebuild the structure
    if thisXoverNode.parent is None:
        thatC = Tree(thisXoverNode, thisC.depth + thatC.depth)
    else:
        thatC.GenStruct()
    if thatXoverNode.parent is None:
        thisC = Tree(thatXoverNode, thisC.depth + thatC.depth)
    else:
        thisC.GenStruct()

    return thisC, thatC

def Crossover(population, parents, probXover):
    '''
    Performs tree crossover on the current generation of a GP after the solutions
    have been selected and paired for mating. Crossover is selected to occur
    with (probXover)% probability - if no crossover, the results are genetic replicates.
    For single-and double-point crossover, the points are selected uniformly.
    :param population: array_like of the current population
    :param parents: (n/2,2) array indicating pairs of solutions to mate; if n is odd,
        parents is of size ((n-1)/2,2)
    :param probXover: scalar float probability of crossover (in range [0,1])
    :return newPop: (n,p) array of next generation's population
    '''
    
    # initialize the container for the new population
    newPop = []
    
    # iterate over pairs of trees
    for pair in parents:
        # get the parents
        this = population[pair[0]]
        that = population[pair[1]]
        if probXover > np.random.rand():
            # cross them over
            offspring = TreesCrossover(this, that)
            newPop.extend(offspring)
        else:
            # just copy them
            newPop.extend((copy.deepcopy(this), copy.deepcopy(that)))
    
    return np.array(newPop)

In [ ]:
def TreeMutate(this, maxDepth, nodeMeta, verbose=False):
    '''
    Mutate a tree at a randomly-selected node.
    :param this: tree to mutate
    :param maxDepth: integer maximum depth allowed for the tree (including the root)
    :param nodeMeta: dictionary holding the a tuple of a list of the node values
        allowed, the number of node values allowed, and node weight for random
        selection; keys are node types of 'ops, 'feat', and 'const'
    :param verbose: optional (default = false) flag to print some info
    :return this_new: the mutated tree
    '''
    # first create a copy of the input tree
    thisC = copy.deepcopy(this)
    
    # get the random mutation point
    thisMutePoint = np.random.permutation(list(chain.from_iterable(thisC.struct.values())))[0]
    if verbose:
        print('This mutation point: %s '%thisMutePoint)
    
    # mutating this node can have big consequences; potentially as big as creating
    # a new tree - so just do that; but it should be shorter than a full tree
    muty = BuildTree(np.random.randint(1, maxDepth), nodeMeta, verbose)
    if verbose:
        print(muty)
    
    # mutate the input tree
    if thisMutePoint.parent is None:
        # mutation point is the root, so just pass the new tree
        thisC = muty
    else:
        muty = muty.root
        # now graft in the new tree
        if thisMutePoint.parent.right == thisMutePoint:
            thisMutePoint.parent.setRight(muty)
        elif thisMutePoint.parent.left == thisMutePoint:
            thisMutePoint.parent.setLeft(muty)
        muty.parent = thisMutePoint.parent
        thisC.GenStruct()

    return thisC

def TreePrune(this, nodeMeta, verbose=False):
    '''
    Prune a tree at a randomly-selected node.
    :param this: tree to prune
    :param nodeMeta: dictionary holding the a tuple of a list of the node values
        allowed, the number of node values allowed, and node weight for random
        selection; keys are node types of 'ops, 'feat', and 'const'
    :param verbose: optional (default = false) flag to print some info
    :return this_new: the pruned tree
    '''
    # first create a copy of the input tree
    thisC = copy.deepcopy(this)
    
    # get the random pruning point - find the first non-root op node
    for thisPrunePoint in np.random.permutation(list(chain.from_iterable(thisC.struct.values()))):
        if (thisPrunePoint.type == 'op') and (thisC.root != thisPrunePoint):
            if verbose:
                print('This pruning point: %s '%thisPrunePoint)
            break
            
    # tree is too short to have a non-root op node, so just do nothing
    if thisPrunePoint == thisC.root:
        pass
    else:
        # create a replacement non-op node
        noOpsK = [k for k in nodeMeta.keys() if k != 'op']
        noOpsW = [nodeMeta[t][2] for t in noOpsK]
        nodeType, _ = RandomWeightedSelect(noOpsK, noOpsW, 0)
        nodeValu = nodeMeta[nodeType][0][np.random.randint(nodeMeta[nodeType][1])]
        muty = Node(nodeType, nodeValu, None)

        # prune the tree
        if thisPrunePoint.parent.right == thisPrunePoint:
            thisPrunePoint.parent.setRight(muty)
        elif thisPrunePoint.parent.left == thisPrunePoint:
            thisPrunePoint.parent.setLeft(muty)
        muty.parent = thisPrunePoint.parent
        thisC.GenStruct()

    return thisC

def MutatePrune(population, prob, maxDepth, nodeMeta, mutePrune):
    '''
    Perform random mutation or pruning on a population of trees, this would
    usually be performed after the previous generation has mated and produced
    the next generation.
    :param population: array_like of the mated population
    :param prob: scalar float probability of mutation or pruning (in range [0,1])
    :param maxDepth: integer maximum depth allowed for the tree (including the root)
    :param nodeMeta: dictionary holding the a tuple of a list of the node values
        allowed, the number of node values allowed, and node weight for random
        selection; keys are node types of 'ops, 'feat', and 'const'
    :param mutePrune: 'm' for mutation, 'p' for pruning
    :return newPop: array of mutated population
    '''
    
    # initialize the container for the new population
    n = len(population)
    newPop = np.array([None]*n)
    
    # get the index of the trees that will mutate
    mutators = (prob > np.random.rand(n))
    
    # save the non-mutators
    newPop[~mutators] = population[~mutators]
    
    # iterate over the trees to mutate and save
    for muteMe in np.nonzero(mutators)[0]:
        if mutePrune == 'm':
            newPop[muteMe] = TreeMutate(population[muteMe], maxDepth, nodeMeta)
        elif mutePrune == 'p':
            newPop[muteMe] = TreePrune(population[muteMe], nodeMeta)
    
    return newPop

In [ ]:
def MateSelect(popFitness, optimGoal, meth):
    '''    
    Generate an index array into the population showing which members to mate.
    If the population size is uneven, the solutions will all be mated as instructed,
    but then at the end, one of the mating pairs will be randomly culled from mating.
    The random selection is proportional to the pairs' average fitnesses.
    :param popFitness: (n,) data array_like of fitness scores from a population of size n
    :param optimGoal: 1 = maximize, -1 = minimize
    :param meth: 1 = sorted, 2 = roulette
    :return parents: (n/2,2) array indicating pairs of solutions to mate; if n is odd,
        parents is of size ((n-1)/2,2)
    '''
    
    # duck pop_fitness into 1d array JAH 20120920
    popFitness = np.array(popFitness, ndmin=1, copy=False)
    populSize = popFitness.size

    # sort fitness scores, if optimGoal is positive, this will sort the scores
    # descending, with the lowest at the front, if optimGoal is negative, it is
    # essentially sorting ascending, with the largest at the front, either way,
    # the best chromosomes are associated with largest roulette bins
    stdIndex = np.argsort(popFitness*-optimGoal)

    # do the work
    if meth == 1:
        # simply mate pairwise
        if populSize % 2 == 1:
            # odd number, so reinsert the best; we want to insert as 3rd so best doesn't mate with self
            stdIndex = np.insert(stdIndex, 2, stdIndex[0])
            populSize += 1
        parents = np.reshape(stdIndex, (populSize//2, 2))
    else:
        # roulette method
        # prepare bins for roulette - bigger bins at the beginning with lower scores
        bins = np.cumsum(np.linspace(populSize, 1, populSize)/(populSize*(populSize + 1.0)/2))
        # first n random numbers to each bin to find each bin that is a lower bound for each rand
        rands_in_bins = np.repeat(rnd.rand(populSize), populSize) >= np.tile(bins, populSize)
        # summing all lower bound flags for each random gives the bin it falls into (since 0-based)
        newPop = np.sum(np.reshape(rands_in_bins, [populSize]*2), axis=1)
        # now index into the stdindex to get parents
        parents = stdIndex[newPop]
        # odd number, so reinsert the best; will randomly permute order, so doesn't matter where
        if populSize % 2 == 1:
            parents = np.insert(parents, 0, stdIndex[0])
            populSize += 1
        # randomly resort then pair up
        parents = np.reshape(parents[rnd.permutation(populSize)], (populSize/2, 2))

        # 20160225 JAH don't want an uneven population size (often from Elitism) to result in
        # such fast population growth, so randomly cull one mating pair, with frequency relative
        # to their average scores (worst avg score is most likely to be culled)
        if popSitness.size % 2 == 1:
            # compute the parent's avg scores, then get the best<>worst sorted 1-based indexes
            srtAvgScs = np.argsort(np.sum(popFitness[parents], axis = 1)*(-optimGoal))+1
            # create the (0,1] bin upper bounds
            cumProbs = np.cumsum(srtAvgScs/np.sum(srtAvgScs))
            # pick which mating pair is culled - it's the last bin upper bound that's <= the random
            cullMe = (srtAvgScs[rnd.rand() <= cumProbs])[0] - 1
            parents = np.hstack((parents[:cullMe], parents[(cullMe+1):]))
            
    return parents

In [ ]:
def RunGP(params, data, objective, nodeMeta, verbose=False, randSeed=None):
    '''
    Run the GP algorithm for a specified dataset. Parameters for the objective
    function must be passed, along with their names in the objective. While
    only params['showTopSubs'] results will be display, the unique best solutions
    from all generations will be returned.
    :param params: dictionary of GP parameters:
        'showTopSubs': integer number of best solutions to show
        'populSize': integer population size
        'numGens': integer number of generations
        'noChangeTerm': integer number generations with insufficient
            improvement before early termination
        'convgcrit': float convergence criteria
        'elitism': True = on, False = off
        'matetype': 1 = sorted, 2 = roulette
        'probXover': float probability of crossover
        'probMutate': float probability of mutation
        'probPrune': float probability of tree pruning
        'optimGoal': 1 = maximize, -1 = minimize
        'plotFlag': True = on, False = off
        'printFreq': integer number of generations by which the GP will print
            progress
        'maxDepth': integer maximum depth allowed for trees (including the root)
    :param data: dictionary expected to hold two items:
        'data': pandas dataframe of data; columns should be X0, X1, ...
        'name': name (descriptive or perhaps file) of data
    :param objective: dictionary of objective function parameters:
        'function': string function to execute whatever modeling is required and
            return the objective score; it can return multiple items, but the
            first must be the score; if a single item is returned, it should be
            in a tuple
        'arguments': dictionary of arguments to pass to the objective function;
            should include at least 'data' and 'tree'
    :param nodeMeta: dictionary holding the a tuple of a list of the node values
        allowed, the number of node values allowed, and node weight for random
        selection; keys are node types of 'op', 'feat', and 'const'
    :param verbose: optional (default = false) flag to print extra info
    :param randSeed: optional (default = none) seed for randomizer; if not passed,
        this will be generated and printed
    :return bestTree: the best tree overall
    :return bestScore: the score of the best overall tree
    :return genBest: array of the best tree from each generation
    :return genScores: array of the best score and the average (of finite) scores
        from each generation
    :return randSeed: the random seed used
    :return tstamp: string timestamp of the GP run
    :return fig: plotly figure of the GP progress
    '''

    # start time and timestamp
    stt = dt.datetime.now()
    sttT = time.perf_counter()
    tstamp = re.sub('[^0-9]', '', stt.isoformat()[:19])
    
    # parse GP parameters
    populSize = int(params['populSize'])
    numGens = int(params['numGens'])
    noChangeTerm = int(params['noChangeTerm'])
    convgCrit = float(params['convgCrit'])
    elitism = bool(params['elitism'])
    probXover = float(params['probXover'])
    mateType = int(params['mateType'])
    probMutate = float(params['probMutate'])
    optimGoal = int(params['optimGoal'])
    plotFlag = bool(params['plotFlag'])
    printFreq = int(params['printFreq'])
    maxDepth = int(params['maxDepth'])
    showTopSubs = int(params['showTopSubs'])
    probPrune = float(params['probPrune'])
    
    # parse the data
    dataName = data['name']
    data = data['data']
    n, p = data.shape
    
    # parse the objective
    objFunc = objective['function']
    objArgs = objective['arguments'] 
    objArgs['data'] = data
    
    # set the random state
    if randSeed is None:
        randSeed = int(str(time.time()).split('.')[1])
        print('Random Seed = %d'%randSeed)
    np.random.seed(randSeed)
    
    # display parameters
    dispLine = '#'*42
    print('%s\nGP Started on %s\n%s'%(dispLine, stt.isoformat(), dispLine))
    print('Data: %s(n=%d, p=%d)'%(dataName, n, p))
    print('Random Seed: %d'%randSeed)
    print('Maximum # Generations: %0.0f\nMininum # of Generations: %0.0f\nConvergence Criteria: %0.8f'%(numGens,noChangeTerm,convgCrit))
    if populSize % 2 == 1:
        populSize += 1
        print('!!Population Size Increased By 1 to be Even!!')
    print('Population Size: %0.0f'%populSize)
    print('Mutation Rate: %0.2f\nPrune Rate: %0.2f\nCrossover Rate: %0.2f'%(probMutate, probPrune, probXover))
    print('Mating Method: %s'%['SORTED','ROULETTE'][mateType - 1])
    print('Elitism is: %s'%['OFF','ON'][elitism])
    print(dispLine)
    if optimGoal == 1:
        print('Objective: MAXIMIZE')
    else:
        print('Objective: MINIMIZE')
    print('Objective Function: %s(%s)'%(objFunc, objArgs.keys()))
    print(dispLine)
    
    # randomly initialize the population of trees
    population = np.array([None]*populSize)
    for indx in range(populSize):
        population[indx] = BuildTree(maxDepth, nodeMeta, True)
        if verbose:
            print('%0d\n%s'%(indx, population[indx]))
    
    # now initialize more things
    # save results by generation
    genScores = np.zeros((numGens, 2), dtype=float)
    genBest = np.array([None]*numGens)
    # current generation's best
    bestTree = None
    bestScore = optimGoal*-1*np.Inf
    # generations with no improvement termination counter
    termCount = 0
    
    # Begin GP Algorithm Whoo Hoo!
    for genCnt in range(numGens):       
        ''' compute or lookup objective function values '''
        popFitness = np.ones(populSize, dtype=float)*np.Inf
        for popCnt in range(populSize):
            if genCnt > 0:
                # check if this tree already evaluated, to save time
                prevEval = np.nonzero(population[popCnt].function == allTrees)[0]
                if prevEval.size == 0:
                    # evalaute
                    objArgs['tree'] = population[popCnt].function
                    popFitness[popCnt] = globals()[objFunc](**objArgs)[0]
                else:
                    # look up existing score
                    popFitness[popCnt] = allScores[prevEval]
            else:
                objArgs['tree'] = population[popCnt].function
                popFitness[popCnt] = globals()[objFunc](**objArgs)[0]
                
        # If optimGoal is (+), this will not change the scores, so the true max will be taken.
        # If optimGoal is (-), the signs will all be changed, and since max(X) = min(-X),
        # taking the maximum will really be taking the minimum.
        optInd = np.argmax(optimGoal*popFitness)
        optVal = popFitness[optInd]
        
        # save some stuff before moving along
        genScores[genCnt,:] = (optVal, np.mean(popFitness[np.isfinite(popFitness)]))
        genBest[genCnt] = population[optInd]
        
        ''' save all unique trees & their scores '''
        if genCnt == 0:
            # first generation, so create these arrays here
            allScores = popFitness
            allTrees = [tree.function for tree in population]
        else:
            # not first generation, so append to the existing arrays
            allScores = np.append(allScores, popFitness)
            allTrees = np.hstack((allTrees, [tree.function for tree in population]))
        # now just get the unique trees
        uniq, ind = np.unique(allTrees, return_index=True)
        allScores = allScores[ind]
        allTrees = uniq
        
        ''' check for early termination '''
        if optimGoal*genScores[genCnt, 0] > optimGoal*bestScore:
            # this is a better score, so save it and reset the counter
            bestScore = genScores[genCnt, 0]
            bestTree = genBest[genCnt]
            termCount = 1
        #elif (optimGoal*genScores[genCnt, 0] < optimGoal*bestScore) and (elitism == False):
        #    # if elitism is off, we can still do early termination with this
        #    termcount += 1
        elif abs(optimGoal*genScores[genCnt, 0] - optimGoal*bestScore) < convgCrit:
            # "no" improvement
            termCount += 1
        elif (elitism  == True):
            # with elitism on and a deterministic objective, performance is monotonically non-decreasing
            termCount += 1

        if termCount >= noChangeTerm:
            print('Early Termination On Generation %d of %d'%(genCnt + 1, numGens))
            genScores = genScores[:(genCnt + 1), :] # keep only up to genCnt spaces (inclusive)
            genBest = genBest[:(genCnt + 1)]
            break
            
        # don't bother with the next generation
        if genCnt == (numGens - 1):
            break
    
        ''' create the next generation '''
        # select parents for the next generation
        parents = MateSelect(popFitness, optimGoal, mateType)
        # mate them
        newPop = Crossover(population, parents, probXover)
        # tree pruning
        newPop = MutatePrune(newPop, probMutate, maxDepth, nodeMeta, 'p')
        # tree mutation
        newPop = MutatePrune(newPop, probMutate, maxDepth, nodeMeta, 'm')

        ''' finalize new population & convey best individual into it if not already there '''
        if elitism:
            # check if best is currently in newPop
            if np.nonzero(bestTree == newPop)[0].size == 0:
                population = np.hstack((newPop, bestTree))
            else:
                population = newPop.copy()
        else:
            population = newPop.copy()
        # readjust in case population grew
        populSize = len(population)
    
        # talk, maybe
        if genCnt % printFreq == 0:
            print('Generation %d of %d: Best Score = %0.4f, Early Termination = %d\n\t%s'%\
                (genCnt + 1, numGens, bestScore, termCount, bestTree.function))
            
    # Finish GP Algorithm Whoo Hoo!
    print('Generation %d of %d: Best Score = %0.4f, Early Termination = %d\n\t%s'%\
        (genCnt + 1, numGens, bestScore, termCount, bestTree.function))
    
    ''' simplify the best trees '''
    print('Simplifying Generation Best Trees')
    bestTree.Simplify()
    for tree in genBest:
        tree.Simplify()
    
    ''' plot GP progress '''
    fig = plysub.make_subplots(rows=3, cols=1, print_grid=False, subplot_titles=['Best Score', 'Average Score', 'Solution Length'])
    fig.add_trace(go.Scatter(x=list(range(len(genBest))), y=genScores[:,0], mode='markers+lines', name='Best Score'), 1, 1)
    fig.add_trace(go.Scatter(x=list(range(len(genBest))), y=genScores[:,1], mode='markers+lines', name='Average Score'), 2, 1)
    fig.add_trace(go.Scatter(x=list(range(len(genBest))), y=[len(tree.function) for tree in genBest], mode='markers+lines', name='Solution Length'), 3, 1)
    fig.update_layout(title='GP Progress Resuls by Generation (%s, %s, %s)'%(dataName, objFunc, tstamp))
    fig['layout']['xaxis3'].update(title='Generation')
    if plotFlag:
        plyoff.plot(fig, filename='../output/GPProgress_%s_%s_%s.html'%(tstamp, re.sub('[^0-9A-Za-z_]', '_', dataName), objFunc), auto_open=True, include_mathjax='cdn')
    
    ''' summaryize results: GA_BEST '''
    # build the results dataframe
    tmp = np.array([[tree.function for tree in genBest], genScores[:,0]]).T
    GA_BEST = pd.DataFrame(data=tmp, columns=['Tree Function', 'Tree Score'])
    # compute the tree bests frequencies, add them in
    freqs = pd.DataFrame(data=GA_BEST['Tree Function'].value_counts()/len(genBest)).reset_index()\
        .rename(columns={'Tree Function':'Frequency', 'index':'Tree Function'})
    GA_BEST = GA_BEST.merge(freqs, on=['Tree Function'], how='inner')
    # drop duplicates and move the function to the index
    GA_BEST = GA_BEST.drop_duplicates().set_index('Tree Function')
    # sort by a temporary column so the best are at the top
    GA_BEST['tmp'] = -1*optimGoal*GA_BEST['Tree Score']
    GA_BEST = GA_BEST.sort_values(by='tmp').drop(columns='tmp', inplace=False)
    
    # show results
    print('%s\nGP Complete\n\tUnique Trees Evaluated - %d\nTop %d Solutions'%(dispLine, len(allScores), showTopSubs))
    display(GA_BEST.head(showTopSubs))
    
    # stop time
    stp = dt.datetime.now()
    stpT = time.perf_counter()
    print('GP: Started on %s\n\tFinished on %s\n\tElapsed Time = %0.3f(m)'%(stt.isoformat(), stp.isoformat(), (stpT-sttT)/60))
    
    return bestTree, bestScore, genBest, genScores, randSeed, tstamp, fig

## Apply some Sample Trees to a Dataframe
<a id=sample></a>
<a href=#top>Go to Top</a>

In [ ]:
# set the possible node values
ops = ['ad', 'sb', 'ml', 'dv', 'pw', 'mx', 'mn']
feats = ['X%d'%i for i in range(5)]
consts = [0, 1, 2, 3, 10, 100]

# must be orderd by descending weight - [values, length, weight] 
nodeMeta = OrderedDict()
nodeMeta['op'] = [ops, len(ops), 0.5]
nodeMeta['feat'] = [feats, len(feats), 0.25]
nodeMeta['const'] = [consts, len(consts), 0.25]

In [ ]:
''' randomly generate some trees '''
# set the prng seed
randSeed = int(str(time.time()).split('.')[1])
print('Random Seed = %d'%randSeed)
np.random.seed(randSeed)

# set the depth
maxDepth = 10

# build the tree, starting from the top node
treeCnt = 20
trees = [None]*treeCnt
for indx in range(treeCnt):
    print('Creating tree %0d'%indx)
    time.sleep(np.random.rand()) # setting a random wait time to allow seed differentiation
    trees[indx] = BuildTree(maxDepth, nodeMeta, True)
    print(trees[indx])

In [ ]:
''' try some GP operations '''
# crossover 2 pairs
trees.extend(TreesCrossover(trees[8], trees[14], True))
trees.extend(TreesCrossover(trees[15], trees[16], True))

# mutate 2
trees.append(TreeMutate(trees[18], maxDepth, nodeMeta, True))
trees.append(TreeMutate(trees[19], maxDepth, nodeMeta, True))

In [ ]:
print(trees[13])
print('--------')
print(trees[16])
print('--------')
print(trees[-4])
print('--------')
print(trees[-3])

In [ ]:
# generate some data
p = len(feats)
n = 1000
data = pd.DataFrame(data=np.random.rand(n, p), columns=feats)
display(data.head())

In [ ]:
# now apply all trees
for indx in range(len(trees)):
    print('Processing tree %0d'%indx)
    func = trees[indx].GenFunction()
    data['tree%0d'%indx] = eval(func.replace('X', 'data.X'))
# talk
display(data.head())

## Apply the GP to a Test dataset
<a id=test></a>
<a href=#top>Go to Top</a>

In [3]:
''' generate some data '''
np.random.seed(42)
p = 5
n = 100

# generate the features
X = np.random.normal(loc=10, scale=1, size=(n,p))
# generate the response
actFunc = 'dv(sb(X0, ml(10, X1)), X2)'
y = (X[:, 0] - 10*X[:, 1])/X[:, 2]# +  np.random.normal(loc=0, scale=0.5, size=(n,))

# build the dataframe
data = pd.DataFrame(data=X, columns=['X%d'%i for i in range(p)])
data['target'] = y

# talk
display(data.head())

,X0,X1,X2,X3,X4,target
0,10.496714,9.861736,10.647689,11.523030,9.765847,-8.276035
1,9.765863,11.579213,10.767435,9.530526,10.542560,-9.846938
2,9.536582,9.534270,10.241962,8.086720,8.275082,-8.377898
3,9.437712,8.987169,10.314247,9.091976,8.587696,-7.798337
4,11.465649,9.774224,10.067528,8.575252,9.455617,-8.569789


In [ ]:
# define an objective function
def RegressionRMSE(data, tree):
    '''
    For a given dataset and tree, evalute the tree on the
    dataset, and assess the linear relationship between the
    results and the target in the data.
    :param data: dataframe of data; columns should include
        'target', and 'X0', 'X1', ...
    :param tree: evaluateable string function of a tree
    :return RMSE: single-element tuple holding RMSE from a
        linear fit between the tree results and the target data column
    '''
    # evaluate the tree function
    treeRes = eval(tree.replace('X', 'data.X'))
    if type(treeRes) is pd.Series:
        # if the tree just encodes a series, have to get values
        treeRes = treeRes.values
    try:
        treeRes = treeRes.reshape(-1, 1)
    except AttributeError:
        # if the tree just encodes a constant value, make an array
        treeRes = np.array([treeRes]*len(data)).reshape(-1, 1)
    
    # regression between target and the tree results
    try:
        linReg = LinearRegression()
        linReg.fit(X=treeRes, y=data['target'].values)
        preds = linReg.predict(X=treeRes)
        RMSE = mean_squared_error(y_true=data['target'].values, y_pred=preds, squared=False)
    except ValueError:
        # nans or infs in the tree res, so just pass out np.inf for RMSE
        RMSE = np.inf
    
    return (RMSE,)

# define an objective function
def TreeRMSE(data, tree):
    '''
    For a given dataset and tree, evalute the tree on the
    dataset, and compute the RMSE between the target and the
    tree results data.
    :param data: dataframe of data; columns should include
        'target', and 'X0', 'X1', ...
    :param tree: evaluateable string function of a tree
    :return RMSE: single-element tuple holding RMSE between
        the tree results and the target data column
    '''
    # evaluate the tree function
    treeRes = eval(tree.replace('X', 'data.X'))
    if type(treeRes) is pd.Series:
        # if the tree just encodes a series, have to get values
        treeRes = treeRes.values
    try:
        treeRes = treeRes.reshape(-1, 1)
    except AttributeError:
        # if the tree just encodes a constant value, make an array
        treeRes = np.array([treeRes]*len(data)).reshape(-1, 1)
    
    # compute RMSE
    try:
        RMSE = mean_squared_error(y_true=data['target'].values, y_pred=treeRes, squared=False)
    except ValueError:
        # nans or infs in the tree res, so just pass out np.inf for RMSE
        RMSE = np.inf
    
    return (RMSE,)

In [ ]:
''' prepare GP input parameters '''
# GP parameters
parmsGP = {'showTopSubs':5, 'populSize':100, 'numGens':200, 'noChangeTerm':200, 'convgCrit':0.1,
           'elitism':True, 'mateType':1, 'probXover':0.7, 'probMutate':0.2, 'optimGoal':-1,
           'plotFlag':True, 'printFreq':10, 'maxDepth':4, 'probPrune':0.4}
# data parameters
parmsData = {'data':data, 'name':'Simulated: %s'%actFunc}
# objective parameters
parmsObj = {'function':'TreeRMSE', 'arguments':{'data':None, 'tree':None}}

# set the possible node values
ops = ['ad', 'sb', 'ml', 'dv', 'pw', 'mx', 'mn']
feats = ['X%d'%i for i in range(p)]
consts = [0, 1, 2, 3, 10, 100]
nodeMeta = OrderedDict() # must be orderd by descending weight - [values, length, weight] 
nodeMeta['op'] = [ops, len(ops), 0.5]
nodeMeta['feat'] = [feats, len(feats), 0.25]
nodeMeta['const'] = [consts, len(consts), 0.25]

In [ ]:
# run the GP - hold on to your butts
randSeed = None#42
verb = False

# ignore all warnings - may be a very bad idea
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    bestTree, bestScore, genBest, genScores, randSeed, timeStamp, figGPProgress = RunGP(parmsGP, parmsData, parmsObj, nodeMeta, verb, randSeed)

In [ ]:
''' evaluate the tree predictions '''
# choose the tree
tree = bestTree
#tree = genBest[50]
print(tree)

# add the tree results
data['tree'] = eval(tree.function.replace('X', 'data.X'))
# compute error
data['error'] = data['target'] - data['tree']

# talk
display(data.head())

# plot
figGPPerformance = ResultsPlots(data, sequenceCol=None, responseCol='target',
                                predCol='tree', resdCol='error', colorCol=None,
                                overall_title='GP Performance', plot_colors=('red',)*4)
plyoff.plot(figGPPerformance, filename='../output/GPPerformance_%s_%s_%s.html'\
            %(timeStamp, re.sub('[^0-9A-Za-z_]', '_', actFunc), parmsObj['function']), auto_open=True, include_mathjax='cdn')

<a href=#top>Go to Top</a>